In [4]:
import pandas as pd

from transformers import AutoTokenizer, AutoModel
import torch

/home/julita/anaconda3/envs/anote/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../Adata/ag_news/train_subset.csv')

In [3]:
df

,label,text
0,Business,Coke is it: BHP coal to double BHP Billiton pl...
1,Business,BIG HIKE IN ENERGY BILLS The company says the ...
2,Business,Officials Blame Contractors in Tunnel Leak The...
3,Business,Before the Bell: Krispy Kreme Down 11 Pct Shar...
4,Sports,Medical Examiner Finds No Injuries on Thanou-S...
...,...,...
995,World,Indian politician sent to prison Top Hindu nat...
996,Sports,Ravens - Bills: Game Notes COMPOSED: With Lewi...
997,Sci/Tech,Dell touts in-home services Dell offers to ins...
998,Sports,"Lately, it's been fast times World Series anal..."


In [5]:
model_name = "dkleczek/bert-base-polish-uncased-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [ ]:
def get_embedding(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output = model(**tokens)
    embeddings = output.last_hidden_state
    mask = tokens['attention_mask'].unsqueeze(-1).expand(embeddings.size()).float()
    masked_embeddings = embeddings * mask
    summed = torch.sum(masked_embeddings, 1)
    counts = torch.clamp(mask.sum(1), min=1e-9)
    mean_pooled = summed / counts
    return mean_pooled.squeeze().numpy()

In [6]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Wybierz angielski model BERT
model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Funkcja: przekształć tekst w embedding przy użyciu mean pooling
def get_mean_pooled_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)

    # Embeddingi tokenów (last hidden state)
    token_embeddings = outputs.last_hidden_state  # [1, seq_len, hidden_size]
    attention_mask = inputs['attention_mask']     # [1, seq_len]

    # Zastosuj maskę uwagi do ignorowania paddingu
    mask = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    masked_embeddings = token_embeddings * mask

    # Średnia z aktywnych tokenów
    summed = torch.sum(masked_embeddings, dim=1)
    counts = torch.clamp(mask.sum(dim=1), min=1e-9)
    mean_pooled = summed / counts

    return mean_pooled.squeeze().numpy()  # Zwraca wektor np. (768,)

# 🔍 Przykład:
text = "This is an example of English text embedding."
embedding = get_mean_pooled_embedding(text)
print(embedding.shape)  # np. (768,)


(768,)


In [7]:
embedding 

array([ 5.31881750e-01,  2.42027432e-01,  1.85968056e-01, -7.63323456e-02,
       -1.52500430e-02,  5.54210581e-02, -3.43517214e-02,  2.19524592e-01,
       -5.10819964e-02, -6.20043993e-01, -2.60024369e-01,  3.65193576e-01,
       -1.57380491e-01,  5.80371976e-01, -8.88942003e-01, -2.78773397e-01,
        6.12197369e-02,  1.01635218e-01,  1.97904017e-02, -3.12814206e-01,
        6.81877211e-02, -2.78766632e-01,  5.27175702e-02, -3.44978511e-01,
        3.05154115e-01, -4.71407771e-01,  2.72142947e-01,  5.72911918e-01,
       -1.42172486e-01,  1.89334691e-01, -7.16156363e-02, -5.27265444e-02,
        2.62595654e-01, -7.23674223e-02, -1.05480790e-01,  8.66547320e-03,
       -7.38206357e-02,  1.92063257e-01, -1.46397591e-01,  2.47650705e-02,
        1.72781140e-01,  1.01263456e-01,  8.71964693e-02, -1.56145498e-01,
        4.22575980e-01, -3.55272472e-01, -1.40996769e-01, -5.62825352e-02,
       -2.34713897e-01,  1.51386604e-01,  3.65764238e-02, -8.52356702e-02,
        9.25283581e-02,  